In [ ]:
import cv2
import numpy as np
from utils.object_detector import ObjectDetector
from utils.template_matcher import TemplateMatcher

In [ ]:
detector = ObjectDetector()
matcher = TemplateMatcher()

In [ ]:
if "google.colab" in str(get_ipython()):
    from google.colab.patches import cv2_imshow

    imshow = cv2_imshow
else:

    def imshow(img):
        cv2.imshow("ImageWindow", img)
        cv2.waitKey()

        cv2.destroyAllWindows()


In [ ]:
img1  = cv2.imread('resources/img1.jpg')
img2  = cv2.imread('resources/board_image2.jpg')
img1 = cv2.resize(img1, None, fx=0.2, fy=0.2)
# img2 = cv2.resize(img2, None, fx=0.2, fy=0.2)

In [ ]:
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

## Helper functions

## Detecting objects on a photo

In [ ]:
detected_objects, detected_cords = detector.detect_objects(img2)

### Idea is to create templates for matching different objects, so far:
 - Capitol
 - Any non reversed card (template for a card is just a random card)

In [ ]:
classes = ['Capitol', 'Opponent Capitol', 'Unit', 'Opponent Unit', 'Support', 'Opponent Support', 'Deck', 'Opponent Deck', 'Deck Shadow', 'Opponent Deck Shadow', 'Barrel', 'Opponent Barrel']
# classes = ['Capitol', 'Unit', 'Support', 'Deck']

In [ ]:
capitol= cv2.imread('resources/capitol_template.png')
capitol_opp = cv2.imread('resources/capitol_opp_template.png')
unit = cv2.imread('resources/unit_template.png')
unit_opp = cv2.imread('resources/unit_opp_template.png')
support = cv2.imread('resources/support_template.png')
support_opp = cv2.imread('resources/support_opp_template.png')
deck = cv2.imread('resources/deck_template.png')
deck_opp = cv2.imread('resources/deck_opp_template.png')
deck_shadow = cv2.imread('resources/deck_shadow_template.png')
deck_opp_shadow = cv2.imread('resources/deck_opp_shadow_template.png')
barrel = cv2.imread('resources/barrel_template.png')
barrel_opp = cv2.imread('resources/barrel_opp_template.png')
templates = [capitol, capitol_opp, unit, unit_opp, support, support_opp, deck, deck_opp, deck_shadow, deck_opp_shadow, barrel, barrel_opp]
# templates = [capitol, unit, support, deck]
# for template in templates:
#   imshow(template)

### Used templates:
![Capitol template](resources/capitol_template.png)
![Unit template](resources/unit_template.png)
![Support template](resources/support_template.png)
![Deck template](resources/deck_template.png)

## Matching templates

Templates are scaled to size of detected object and then each template is matched to each object to return which objects match given template, instead of hard coded value we would like to explore determining objects by most simmilar template given some small threshold is passed to exclude noise

In [ ]:
# for obj in detected_objects:
#   imshow(obj)

In [ ]:
determined_classes = matcher.match_templates(templates, detected_objects)

In [ ]:
# imshow(matcher.draw_detections(img2, detected_cords, determined_classes, classes))

In [ ]:
def denoise_detections(sequence):
    highest_numbers = []
    current_number = None
    consecutive_count = 0
    highest_number = None

    for number in sequence:
        if number == current_number:
            consecutive_count += 1
        else:
            consecutive_count = 1
            current_number = number

        if consecutive_count >= 2:
            if highest_number is None or current_number > highest_number:
                highest_number = current_number
        highest_numbers.append(highest_number)

    return highest_numbers

In [ ]:
video_path = 'resources/easy2.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Couldn't open video.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_path = 'outputs/video.avi'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_number = 0
process_every_n_frames = 20
display_counter = 999
card_played = None
deck_detected = [False, False]
resources = [0, 0]
cards = {
    'Unit': (0, []),
    'Support': (0, []),
    'Opponent Unit': (0, []),
    'Opponent Support': (0, [])
}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_number % process_every_n_frames == 0:
        detected_objects, detected_cords = detector.detect_objects(frame)

        determined_classes = matcher.match_templates(templates, detected_objects)
        deck_detected_this_frame = [False, False]
        resources = [0, 0]
        cards_this_frame = {
            'Unit': 0,
            'Support': 0,
            'Opponent Unit': 0,
            'Opponent Support': 0
        }

        for obj_class in determined_classes:
            if (np.argmax(obj_class) == 8 or np.argmax(obj_class) == 6) and obj_class[np.argmax(obj_class)] > 0.3:
                deck_detected[0] = True
                deck_detected_this_frame[0] = True
                continue

            if (np.argmax(obj_class) == 9 or np.argmax(obj_class) == 7) and obj_class[np.argmax(obj_class)] > 0.3:
                deck_detected[1] = True
                deck_detected_this_frame[1] = True
                continue

            if np.argmax(obj_class) == 10 and obj_class[np.argmax(obj_class)] > 0.3:
                resources[0] += 1
                continue
            
            if np.argmax(obj_class) == 11 and obj_class[np.argmax(obj_class)] > 0.3:
                resources[1] += 1
                continue

            if np.argmax(obj_class) == 2 and obj_class[np.argmax(obj_class)] > 0.3:
                cards_this_frame['Unit'] += 1
            
            if np.argmax(obj_class) == 3 and obj_class[np.argmax(obj_class)] > 0.3:
                cards_this_frame['Opponent Unit'] += 1
            
            if np.argmax(obj_class) == 4 and obj_class[np.argmax(obj_class)] > 0.3:
                cards_this_frame['Support'] += 1
            
            if np.argmax(obj_class) == 5 and obj_class[np.argmax(obj_class)] > 0.3:
                cards_this_frame['Opponent Support'] += 1

        for card_type, card_num in cards_this_frame.items():
            cards[card_type][1].append(card_num)
          
        if not deck_detected_this_frame[0]:
            deck_detected[0] = False
        if not deck_detected_this_frame[1]:
            deck_detected[1] = False
   
    for card_type, (card_num, sequence) in cards.items():
        if denoise_detections(sequence)[-1] and card_num < denoise_detections(sequence)[-1]:
            if card_num:
                display_counter = 0
                card_played = card_type
                cv2.putText(frame, f"{card_played} Was Played", (50, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,105,180), 2)

            cards[card_type] = (denoise_detections(sequence)[-1], sequence)
            
        if display_counter < 4 * process_every_n_frames:
            display_counter += 1
            cv2.putText(frame, f"{card_played} Was Played", (50, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,105,180), 2)
            break

    cv2.putText(frame, f"Player 1 Resources: {resources[0]}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Player 2 Resources: {resources[1]}", (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    if not deck_detected[0]:
       cv2.putText(frame, "Player 1 Draws Card!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    if not deck_detected[1]:
         cv2.putText(frame, "Player 2 Draws Card!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    frame = matcher.draw_detections(frame, detected_cords, determined_classes, classes)

    out.write(frame)

    frame_number += 1

for k, v in cards.items():
    print(f'{k}: {denoise_detections(v[1])}')

cap.release()
out.release()

cv2.destroyAllWindows()
